In [1]:
from calendar import month
from operator import index
import snscrape.modules.twitter as sntwitter
import pandas as pd
import glob
import datetime

release_dates = pd.read_csv("Release dates - data.csv")
release_dates_map = {}

folderpathrt = "rotten_info/*"
filesrt = []

for file in glob.glob(folderpathrt):
    filesrt.append(file)

for file in filesrt:
    print(file)
    movies = pd.read_csv(file)

    tweets = []
    for ind, movie in movies.iterrows():
        movien = movie['Release Group']
        if not pd.isna(movie['Release Date (Theaters)']):
            release_date = ('').join(movie['Release Date (Theaters)'].strip('[]').split(',')).replace("'", "")
            rd = datetime.datetime.strptime(release_date, "%b %d %Y")
            # if release_dates_map.get(movien.lower(), ''):
            release_dates_map[movien.lower()] = rd

folderpath = "Box Office Mojo/*"
files = []

for file in glob.glob(folderpath):
    files.append(file)

for ind, row in release_dates.iterrows():
    if row['Release-Date']:
        rd = datetime.datetime.strptime(row['Release-Date'], "%Y-%m-%d")
        if not release_dates_map.get(row['Movie'].lower(), None):
            release_dates_map[row['Movie'].lower()] = rd
    
count = 0
c = 0
for file in files:
    if '2022' in file:
        print(file)
        movies = pd.read_csv(file)

        tweets = []
        for ind, movie in movies.iterrows():
            c += 1
            query_txt = movie['Release Group']
            release_date = release_dates_map.get(movie['Release Group'].lower(), None)
            if release_date:
                count += 1
                # rd = datetime.datetime.strptime(release_date, "%Y-%m-%d")
                since = release_date - datetime.timedelta(weeks=4)
                until = release_date + datetime.timedelta(weeks=1)
                query_txt += " since:%s until:%s" % (since.date(), until.date())
                print(movie['Release Group'], release_date.date())

                for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query_txt).get_items()):
                    if i>5000:
                        break
                    tweets.append([movie['Release Group'], tweet.date, tweet.id, tweet.content, tweet.user.username])

        tweets_df = pd.DataFrame(tweets, columns=['Movie Name', 'Datetime', 'Tweet Id', 'Text', 'Username'])
        tweets_df.to_csv("Twitter/tweets_%s" % file.split('/')[1], index=False, escapechar=r'|')
        break


Box Office Mojo/2022.csv
The Batman 2022-03-04
Water Gate Bridge 2022-02-11
Uncharted 2022-02-18
Too Cool to Kill 2022-02-18
Scream 2022-01-14
Death on the Nile 2022-02-11
Jackass Forever 2022-02-04
Dog 2022-02-18
Marry Me 2022-02-11
Moonfall 2022-02-04
BTS Permission to Dance on Stage - Seoul: Live Viewing 2022-03-12
The 355 2022-01-07
The Kashmir Files 2022-03-18
Radhe Shyam 2022-03-11
Blacklight 2022-02-11
Redeeming Love 2022-01-21
Cyrano 2022-02-25
The Cherry Bushido 2022-03-11
The Cursed 2022-02-18
X 2022-03-18
Gangubai Kathiawadi 2022-03-04
Studio 666 2022-02-25
The Beatles: Get Back - The Rooftop Concert 2022-01-30
The King's Daughter 2022-01-21
The Outfit 2022-03-18
Umma 2022-03-18
The Tiger Rising 2022-01-21
The Family 2013-09-13
Tyson's Run 2022-03-11
Clean 2022-01-28
Sundown 2022-01-28
Alice 2022-03-18
Jane by Charlotte 2022-03-18
